**About Book Crossing Dataset**<br>

This dataset has been compiled by Cai-Nicolas Ziegler in 2004, and it comprises of three tables for users, books and ratings. Explicit ratings are expressed on a scale from 1-10 (higher values denoting higher appreciation) and implicit rating is expressed by 0.

Reference: http://www2.informatik.uni-freiburg.de/~cziegler/BX/ 

**Objective**

This project entails building a Book Recommender System for users based on user-based and item-based collaborative filtering approaches.

#### Execute the below cell to load the datasets

In [76]:
import pandas as pd
%matplotlib inline
from sklearn.model_selection import train_test_split
import numpy as np
import time
import Recommenders as Recommenders
from sklearn.externals import joblib
import Evaluation as Evaluation


In [77]:
#Loading data
books = pd.read_csv("books/books.csv", sep=";", error_bad_lines=False, encoding="latin-1")
books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']

users = pd.read_csv('books/users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
users.columns = ['userID', 'Location', 'Age']

ratings = pd.read_csv('books/ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
ratings.columns = ['userID', 'ISBN', 'bookRating']

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.p

### Check no.of records and features given in each dataset

In [78]:
books.shape

(271360, 8)

In [79]:
ratings.shape

(1149780, 3)

In [80]:
users.shape

(278858, 3)

## Exploring books dataset

In [81]:
books.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlS,imageUrlM,imageUrlL
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


### Drop last three columns containing image URLs which will not be required for analysis

In [82]:
books.drop(['imageUrlS','imageUrlM','imageUrlL'],axis=1,inplace=True)

In [83]:
books.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


**yearOfPublication**

### Check unique values of yearOfPublication


In [84]:
books.yearOfPublication.unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984, 0, 1968, 1961, 1958, 1974,
       1976, 1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960,
       1966, 1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954,
       1950, 1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011,
       1925, 1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 2030,
       1911, 1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 2050, 1934,
       1910, 1933, 1902, 1924, 1921, 1900, 2038, 2026, 1944, 1917, 1901,
       2010, 1908, 1906, 1935, 1806, 2021, '2000', '1995', '1999', '2004',
       '2003', '1990', '1994', '1986', '1989', '2002', '1981', '1993',
       '1983', '1982', '1976', '1991', '1977', '1998', '1992', '1996',
       '0', '1997', '2001', '1974', '1968', '1987', '1984', '1988',
       '1963', '1956', '1970', '1985', '1978', '1973', '1980'

As it can be seen from above that there are some incorrect entries in this field. It looks like Publisher names 'DK Publishing Inc' and 'Gallimard' have been incorrectly loaded as yearOfPublication in dataset due to some errors in csv file.


Also some of the entries are strings and same years have been entered as numbers in some places. We will try to fix these things in the coming questions.

### Check the rows having 'DK Publishing Inc' as yearOfPublication

In [85]:
books[books['yearOfPublication'].eq('DK Publishing Inc')]

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
209538,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...
221678,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...


### Drop the rows having `'DK Publishing Inc'` and `'Gallimard'` as `yearOfPublication`

In [86]:
books.drop([209538,221678,220731],axis=0,inplace=True)
books[books['yearOfPublication'].eq('Gallimard')]

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher


### Change the datatype of yearOfPublication to 'int'

In [87]:
books['yearOfPublication'] = books['yearOfPublication'].astype(int)

In [88]:
books.dtypes

ISBN                 object
bookTitle            object
bookAuthor           object
yearOfPublication     int32
publisher            object
dtype: object

### Drop NaNs in `'publisher'` column


In [89]:
books.publisher.dropna(inplace=True)

## Exploring Users dataset

In [90]:
print(users.shape)
users.head()

(278858, 3)


,userID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


### Get all unique values in ascending order for column `Age`

In [91]:
Age=users['Age'].unique()
#users.sort_values(by='Age',ascending=True).unique()
print(sorted(Age))

[nan, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 113.0, 114.0, 115.0, 116.0, 118.0, 119.0, 123.0, 124.0, 127.0, 128.0, 132.0, 133.0, 136.0, 137.0, 138.0, 140.0, 141.0, 143.0, 146.0, 147.0, 148.0, 151.0, 152.0, 156.0, 157.0, 159.0, 162.0, 168.0, 172.0, 175.0, 183.0, 186.0, 189.0, 199.0, 200.0, 201.0, 204.0, 207.0, 208.0, 209.0, 210.0, 212.0, 219.0, 220.0, 223.0, 226.0

Age column has some invalid entries like nan, 0 and very high values like 100 and above

### Values below 5 and above 90 do not make much sense for our book rating case...hence replace these by NaNs

In [92]:
users.Age=users.Age.where(users.Age.between(5,90))
Age=users.Age.unique()
print(sorted(Age))

[nan, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0]


### Replace null values in column `Age` with mean

In [93]:
users.Age.fillna(users.Age.mean(),inplace=True)


### Change the datatype of `Age` to `int`

In [94]:
users['Age'] = users['Age'].astype(int)

In [95]:
print(sorted(users.Age.unique()))

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90]


## Exploring the Ratings Dataset

### check the shape

In [96]:
ratings.shape

(1149780, 3)

In [97]:
n_users = users.shape[0]
n_books = books.shape[0]

In [98]:
ratings.head(5)

,userID,ISBN,bookRating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


### Ratings dataset should have books only which exist in our books dataset. Drop the remaining rows

In [99]:
ratings=ratings.loc[ratings.ISBN.isin(books.ISBN)]
ratings.shape


(1031132, 3)

### Ratings dataset should have ratings from users which exist in users dataset. Drop the remaining rows

In [100]:
ratings=ratings.loc[ratings.userID.isin(users.userID)]
ratings.shape

(1031132, 3)

### Consider only ratings from 1-10 and leave 0s in column `bookRating`

In [101]:
#ratings.bookRating = ratings.bookRating.where(ratings.bookRating.rank(method='min',ascending=False)>10, 0)

### Find out which rating has been given highest number of times

In [102]:
totCount=ratings.groupby('bookRating')['bookRating'].count()
totCount.sort_values(ascending=False)[:10]
#rating 0 has been given highest number of times.

bookRating
0     647291
8      91804
10     71225
7      66401
9      60778
5      45355
6      31687
4       7617
3       5118
2       2375
Name: bookRating, dtype: int64

### **Collaborative Filtering Based Recommendation Systems**

### For more accurate results only consider users who have rated atleast 100 books

In [103]:
rating = ratings.groupby(['userID'])['ISBN'].count()>100
rating=pd.DataFrame(rating)
rating = rating[rating['ISBN']== True]
rating.reset_index(level=0,inplace=True)

In [104]:
ratings=ratings.loc[ratings.userID.isin(rating.userID)]
ratings.shape

(591894, 3)

### Generating ratings matrix from explicit ratings


#### Note: since NaNs cannot be handled by training algorithms, replace these by 0, which indicates absence of ratings

In [105]:
ratings.replace('nan',0,inplace=True)


### Generate the predicted ratings using SVD with no.of singular values to be 50

In [106]:
R_df = ratings.pivot(index = 'userID', columns ='ISBN', values = 'bookRating').fillna(0)
R_df.tail()


ISBN,0000913154,0001010565,0001046438,000104687X,0001047213,0001047647,0001047663,0001047868,0001047973,000104799X,...,B0001GDNCK,B0001GMSV2,B0001I1JII,B0001I1KOG,B0001PBXMS,B0001PIOX4,B000234N3A,B000234N76,B000234NC6,B00029DGGO
userID,,,,,,,,,,,,,,,,,,,,,
277478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
277639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
278137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
278188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
278418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [107]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_df, k = 50)

In [108]:
sigma = np.diag(sigma)


In [109]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

In [110]:
preds_df.head()

ISBN,0000913154,0001010565,0001046438,000104687X,0001047213,0001047647,0001047663,0001047868,0001047973,000104799X,...,B0001GDNCK,B0001GMSV2,B0001I1JII,B0001I1KOG,B0001PBXMS,B0001PIOX4,B000234N3A,B000234N76,B000234NC6,B00029DGGO
0,-0.012982,0.0,-0.004371,-0.002914,-0.004371,0.0,0.0,0.0,-0.004371,-0.001562,...,0.0,0.006450,0.0,-0.001751,0.0,0.0,0.033826,0.0,0.0,0.0
1,-0.002466,0.0,-0.004109,-0.002740,-0.004109,0.0,0.0,0.0,-0.004109,0.004419,...,0.0,0.002900,0.0,0.002387,0.0,0.0,-0.007205,0.0,0.0,0.0
2,-0.008407,0.0,0.000028,0.000019,0.000028,0.0,0.0,0.0,0.000028,0.007427,...,0.0,0.008955,0.0,0.004440,0.0,0.0,0.004196,0.0,0.0,0.0
3,0.001455,0.0,0.033411,0.022274,0.033411,0.0,0.0,0.0,0.033411,0.004558,...,0.0,0.001961,0.0,0.001737,0.0,0.0,0.034971,0.0,0.0,0.0
4,0.003655,0.0,0.022143,0.014762,0.022143,0.0,0.0,0.0,0.022143,0.005438,...,0.0,0.003949,0.0,0.002797,0.0,0.0,0.003384,0.0,0.0,0.0


In [111]:
sigma

array([[150.27248898,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        , 151.34836851,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        , 151.86402584, ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [  0.        ,   0.        ,   0.        , ..., 380.46213392,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
        635.93072868,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        , 680.93175919]])

### Take a particular user_id

### Lets find the recommendations for user with id `2110`

#### Note: Execute the below cells to get the variables loaded

In [112]:
userID = 2110

In [129]:
user_id = 2 #2nd row in ratings matrix and predicted matrix
users.head()

,userID,Location,Age
0,1,"nyc, new york, usa",34
1,2,"stockton, california, usa",18
2,3,"moscow, yukon territory, russia",34
3,4,"porto, v.n.gaia, portugal",17
4,5,"farnborough, hants, united kingdom",34


### Get the predicted ratings for userID `2110` and sort them in descending order

In [132]:
user_row_number = user_id - 1 
num_recommendations=5
sorted_user_predictions = preds_df.iloc[user_id-1].sort_values(ascending=False)

user_data = ratings[ratings.userID == (userID)]
user_full = (user_data.merge(books, how = 'left', left_on = 'ISBN', right_on = 'ISBN')
            #sort_values(['userID'], ascending=False)
                 )
recommendations = (ratings[~ratings['ISBN'].isin(user_full['ISBN'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left', left_on = 'ISBN', right_on = 'ISBN').
         rename(columns = {user_row_number: 'Predictions'}))
    #sort the prediction values
recommendations = (recommendations.sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )


In [144]:
recommendations.head()

,userID,ISBN,bookRating
68452,30716,0316666343,0
203367,98391,0316666343,9
544750,252222,0316666343,0
513867,240035,0316666343,0
201825,97874,0316666343,9


### Create a dataframe with name `user_data` containing userID `2110` explicitly interacted books

In [134]:
user_data.head()

,userID,ISBN,bookRating
14447,2110,0060506989,0
14448,2110,0060987529,7
14449,2110,0064472779,8
14450,2110,0140022651,10
14451,2110,0140298479,0


In [135]:
user_data.shape

(163, 3)

In [136]:
user_data.head()

,userID,ISBN,bookRating
14447,2110,0060506989,0
14448,2110,0060987529,7
14449,2110,0064472779,8
14450,2110,0140022651,10
14451,2110,0140298479,0


In [137]:
user_data.shape

(163, 3)

### Combine the user_data and and corresponding book data(`book_data`) in a single dataframe with name `user_full_info`

In [138]:
user_full.head()

,userID,ISBN,bookRating,bookTitle,bookAuthor,yearOfPublication,publisher
0,2110,0060506989,0,We Were Soldiers Once... and Young: Ia Drang--...,Harold G. Moore,2002,HarperTorch
1,2110,0060987529,7,Confessions of an Ugly Stepsister : A Novel,Gregory Maguire,2000,Regan Books
2,2110,0064472779,8,All-American Girl,Meg Cabot,2003,HarperTrophy
3,2110,0140022651,10,Journey to the Center of the Earth,Jules Verne,1965,Penguin Books
4,2110,0140298479,0,Bridget Jones: The Edge of Reason,Helen Fielding,2001,Penguin Books


In [141]:
user_full.shape

(163, 7)

In [71]:
book_data.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
246,0151008116,Life of Pi,Yann Martel,2002,Harcourt
904,015216250X,So You Want to Be a Wizard: The First Book in ...,Diane Duane,2001,Magic Carpet Books
1000,0064472779,All-American Girl,Meg Cabot,2003,HarperTrophy
1302,0345307674,Return of the Jedi (Star Wars),James Kahn,1983,Del Rey Books
1472,0671527215,Hitchhikers's Guide to the Galaxy,Douglas Adams,1984,Pocket


In [73]:
user_full_info.head()

,userID,ISBN,bookRating,bookTitle,bookAuthor,yearOfPublication,publisher
0,2110,0060987529,7,Confessions of an Ugly Stepsister : A Novel,Gregory Maguire,2000,Regan Books
1,2110,0064472779,8,All-American Girl,Meg Cabot,2003,HarperTrophy
2,2110,0140022651,10,Journey to the Center of the Earth,Jules Verne,1965,Penguin Books
3,2110,0142302163,8,The Ghost Sitter,Peni R. Griffin,2002,Puffin Books
4,2110,0151008116,5,Life of Pi,Yann Martel,2002,Harcourt


### Get top 10 recommendations for above given userID from the books not already rated by that user

### Since create method not giving desired output cant perform furthur task.

In [145]:
user_full.head(10)

,userID,ISBN,bookRating,bookTitle,bookAuthor,yearOfPublication,publisher
0,2110,0060506989,0,We Were Soldiers Once... and Young: Ia Drang--...,Harold G. Moore,2002,HarperTorch
1,2110,0060987529,7,Confessions of an Ugly Stepsister : A Novel,Gregory Maguire,2000,Regan Books
2,2110,0064472779,8,All-American Girl,Meg Cabot,2003,HarperTrophy
3,2110,0140022651,10,Journey to the Center of the Earth,Jules Verne,1965,Penguin Books
4,2110,0140298479,0,Bridget Jones: The Edge of Reason,Helen Fielding,2001,Penguin Books
5,2110,0142302163,8,The Ghost Sitter,Peni R. Griffin,2002,Puffin Books
6,2110,0151008116,5,Life of Pi,Yann Martel,2002,Harcourt
7,2110,0151879834,0,Tales from Margaritaville: Fictional Facts and...,Jimmy Buffett,1989,Harcourt
8,2110,015216250X,8,So You Want to Be a Wizard: The First Book in ...,Diane Duane,2001,Magic Carpet Books
9,2110,0152162577,0,Deep Wizardry: The Second Book in the Young Wi...,Diane Duane,2001,Magic Carpet Books
